# [실습] RAG의 필요성

RAG가 왜 필요한지, RAG를 쓸 때와 안 쓸 때의 차이를 극명하게 비교하여 RAG의 필요성을 체감합니다.

In [1]:
# --- 1. 필요 라이브러리 설치 ---
!pip install openai langchain langchain-openai beautifulsoup4 chromadb tiktoken faiss-cpu sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# --- 2. OpenAI API 키 설정 ---
import os

os.environ["OPENAI_API_KEY"] = "sk-gRqhsT2_U1YFOGYerzzIoOu1gpw0JmMnrXQosCx_4IT3BlbkFJRTMSFduGtHsnUrIP0Bsf3gfjjk4ucCPXv4Fw3lxXoA"

## 1단계: RAG 없이 LLM에게 질문하기
LLM이 학습하지 않았을 법한 최신 (가상) 정보를 만들어 질문해 보기

In [3]:
# LLM 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini")

# LLM에게 가상 제품에 대해 질문
question = "메타의 최신 AI 안경은 뭐야?"

# RAG 없이 LLM의 답변 확인
response_without_rag = llm.invoke(question)
print("--- RAG 미사용 시 답변 ---")
print(response_without_rag.content)

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- RAG 미사용 시 답변 ---
2023년 현재 메타(Meta)는 "메타 글래스"라는 AI 기능이 탑재된 스마트 안경을 개발하고 있습니다. 이 제품은 증강 현실(AR) 기술을 활용하여 실시간 정보 표시, 비디오 촬영 및 공유, 음성 인식 기능 등을 지원합니다. 메타는 이러한 안경을 통해 사용자들이 보다 자연스럽고 편리하게 디지털 정보를 사용할 수 있도록 하는 것을 목표로 하고 있습니다.

정확한 모델 이름이나 추가적인 사양은 메타의 공식 발표나 최신 뉴스에서 확인하는 것이 좋습니다. 메타는 기술 발전에 따라 새로운 기능이나 제품을 지속적으로 개발하고 있으니, 최신 정보를 체크하는 것이 중요합니다.


## 2단계: RAG를 사용하여 LLM에게 질문하기

- 본문 참고 링크 : https://www.news1.kr/it-science/general-it/5917344

In [9]:
meta_product_news = """
메타가 17일(현지시간) '메타커넥트 2025'에서 차세대 스마트글라스 '메타 레이밴 디스플레이'(Meta Ray-Ban Display) '오클리 메타 뱅가드'(Oakley Meta Vanguard) '레이밴 메타 2세대'(Ray-Ban Meta Gen 2) 등 3종을 공개했다.
메타 레이밴 디스플레이는 메타와 레이밴의 협업 라인업 중 최초로 디스플레이를 탑재한 모델로 스마트폰을 꺼내지 않고 △지도 △메시지 △번역 △영상통화 등 다양한 기능을 안경 렌즈 위에서 구현한다.
해상도는 시야각당 42픽셀(PPD), 최대 5000니트 밝기를 구현해 야외에서도 활용할 수 있다.
메타 측은 "기존 스마트안경이 음악 감상이나 영상 촬영, AI 엔진 접속 기능 등에 머물렀다면 신제품은 고해상도 단안형 디스플레이를 적용해 문자 메시지 확인이나 짧은 영상을 시청할 수 있다"고 말했다.
메타는 스포츠 활동에 특화된 '오클리 메타 뱅가드'(Oakley Meta Vanguard)도 함께 공개했다.
오클리 메타 뱅가드는 얼굴 전면을 가리는 '랩어라운드' 형식으로 △카메라 △음악 재생 △통화 △인공지능(AI) 기능 등을 활용할 수 있다.
"""

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. RAG 프롬프트 템플릿 정의
prompt_template = """
당신은 주어진 문맥(context) 정보를 바탕으로 사용자의 질문에 답변하는 AI 어시스턴트입니다.
문맥에 없는 내용은 답변하지 마세요.

[Context]
{context}

[Question]
{question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# 2. RAG 체인 구성 (가장 간단한 형태)
rag_chain = prompt | llm | StrOutputParser()

# 3. RAG 체인 실행
response_with_rag = rag_chain.invoke(
    {"context": meta_product_news, "question": question}  # LLM에게 참고 자료 제공
)

print("\n--- RAG 사용 시 답변 ---")
print(response_with_rag)


--- RAG 사용 시 답변 ---
메타의 최신 AI 안경은 '메타 레이밴 디스플레이'입니다. 이 모델은 디스플레이를 탑재하여 스마트폰을 꺼내지 않고도 다양한 기능을 사용할 수 있으며, 고해상도 단안형 디스플레이를 적용해 문자 메시지 확인이나 짧은 영상을 시청할 수 있습니다.


# RAG 기본 활용

In [10]:
# --- 0. 실습에 필요한 라이브러리 추가 설치 ---
!pip install beautifulsoup4 langchain_chroma langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: pydantic-settings
    Found existing installation: pydantic-settings 2.6.1
    Uninstalling pydantic-settings-2.6.1:
      Successfully uninstalled pydantic-settings-2.6.1



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

# --- 1. 데이터 로딩 (Load) ---
loader = WebBaseLoader(
    web_paths=(
        "https://ko.wikipedia.org/wiki/%EB%A9%94%ED%83%80_%ED%94%8C%EB%9E%AB%ED%8F%BC%EC%8A%A4",
    )
)
docs = loader.load()
print(f"docs: {docs}")


docs: [Document(metadata={'source': 'https://ko.wikipedia.org/wiki/%EB%A9%94%ED%83%80_%ED%94%8C%EB%9E%AB%ED%8F%BC%EC%8A%A4', 'title': '메타 플랫폼스 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}, page_content='\n\n\n\n메타 플랫폼스 - 위키백과, 우리 모두의 백과사전\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문으로 이동\n\n\n\n\n\n\n\n주 메뉴\n\n\n\n\n\n주 메뉴\n사이드바로 이동\n숨기기\n\n\n\n\t\t둘러보기\n\t\n\n\n대문최근 바뀜요즘 화제임의의 문서로\n\n\n\n\n\n\t\t사용자 모임\n\t\n\n\n사랑방사용자 모임관리 요청\n\n\n\n\n\n\t\t편집 안내\n\t\n\n\n소개도움말정책과 지침질문방\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n보이기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n기부\n\n계정 만들기\n\n로그인\n\n\n\n\n\n\n\n\n개인 도구\n\n\n\n\n\n기부 계정 만들기 로그인\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n목차\n사이드바로 이동\n숨기기\n\n\n\n\n처음 위치\n\n\n\n\n\n1\n역사\n\n\n\n\n역사 하위섹션 토글하기\n\n\n\n\n\n1.1\n2018–2020: 메타버스에 대한 집중\n\n\n\n\n\n\n\n\n1.2\n2021: 메타로 리브랜딩\n\n\n\n\n\n\n\n\n1.3\n2022: 수익 감소 및 대량 해고\n\n\n\n\n\n\n\n\n1.4\n2023년 이후: 스레드, AI 및 사

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
당신은 주어진 문맥(context) 정보를 바탕으로 사용자의 질문에 답변하는 AI 어시스턴트입니다.
문맥에 없는 내용은 답변하지 마세요.

[Context]
{context}

[Question]
{question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

In [23]:
# --- 2. 분할 (Split) ---
# 불러온 문서를 의미 있는 단위(Chunk)로 나눕니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

In [24]:
print(splits)

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/%EB%A9%94%ED%83%80_%ED%94%8C%EB%9E%AB%ED%8F%BC%EC%8A%A4', 'title': '메타 플랫폼스 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}, page_content='메타 플랫폼스 - 위키백과, 우리 모두의 백과사전\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문으로 이동\n\n\n\n\n\n\n\n주 메뉴\n\n\n\n\n\n주 메뉴\n사이드바로 이동\n숨기기\n\n\n\n\t\t둘러보기\n\t\n\n\n대문최근 바뀜요즘 화제임의의 문서로\n\n\n\n\n\n\t\t사용자 모임\n\t\n\n\n사랑방사용자 모임관리 요청\n\n\n\n\n\n\t\t편집 안내'), Document(metadata={'source': 'https://ko.wikipedia.org/wiki/%EB%A9%94%ED%83%80_%ED%94%8C%EB%9E%AB%ED%8F%BC%EC%8A%A4', 'title': '메타 플랫폼스 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}, page_content='편집 안내\n\t\n\n\n소개도움말정책과 지침질문방\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n보이기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n기부\n\n계정 만들기\n\n로그인\n\n\n\n\n\n\n\n\n개인 도구\n\n\n\n\n\n기부 계정 만들기 로그인'), Document(metadata={'source': 'https://ko.wikipedia.org/wiki/%EB%A9%94%ED%83%80_%ED%94%8C%EB%9E%AB%ED%

In [25]:
# --- 3. 임베딩 & 저장 (Embed & Store) ---
# 나눈 문서 조각들을 벡터로 변환(임베딩)하고, 검색을 위해 벡터 저장소(Vector Store)에 저장합니다.
# Chroma는 간단한 실습에 사용하기 좋은 인메모리(in-memory) 벡터 저장소입니다.
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [26]:
# --- 4. 검색 (Retrieve) ---
# 사용자의 질문과 의미적으로 가장 유사한 문서 조각을 찾아주는 '검색기(Retriever)'를 생성합니다.
retriever = vectorstore.as_retriever()

In [27]:
# --- 5. RAG 체인 구성 및 실행 ---
# 이전 실습과 동일한 프롬프트와 LLM을 사용합니다.
# retriever가 질문을 받아 관련된 문서(context)를 찾아주는 부분이 추가되었습니다.
real_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
# RAG 체인에 질문을 던져봅니다.
question = "메타의 최신 AI 안경은 뭐야?"
response = real_rag_chain.invoke(question)

print("--- RAG 기본 파이프라인 답변 ---")
print(response)

--- RAG 기본 파이프라인 답변 ---
메타의 최신 AI 안경은 오리온입니다. 2024년 커넥트 컨퍼런스에서 처음 선보였으나, 원래 소비자에게 판매될 예정이었지만 제조 공정이 복잡하고 비용이 많이 드는 것으로 밝혀져, 대신 소수의 안경을 내부용으로 생산할 계획으로 전환했습니다.


In [31]:
# [보너스] 실제로 어떤 내용이 검색되었는지 확인해보기
retrieved_docs = retriever.invoke(question)
print("\n\n--- 검색된 문서 조각 (Context) ---")
for doc in retrieved_docs:
    print(doc.page_content)
    print("-" * 30)



--- 검색된 문서 조각 (Context) ---
2024년 커넥트 컨퍼런스에서 메타는 최초의 증강 현실 안경인 오리온을 선보였다.[89] 오리온은 원래 소비자에게 판매될 예정이었지만, 제조 공정이 너무 복잡하고 비용이 많이 드는 것으로 밝혀졌다.[90] 대신, 회사는 소수의 안경을 생산하여 내부용으로 사용할 계획으로 전환했다.[91]
------------------------------
2024년 커넥트 컨퍼런스에서 메타는 최초의 증강 현실 안경인 오리온을 선보였다.[89] 오리온은 원래 소비자에게 판매될 예정이었지만, 제조 공정이 너무 복잡하고 비용이 많이 드는 것으로 밝혀졌다.[90] 대신, 회사는 소수의 안경을 생산하여 내부용으로 사용할 계획으로 전환했다.[91]
------------------------------
2022년부터 메타는 전문 인공지능 하드웨어 및 소프트웨어 채택에서 다른 기술 기업들을 따라잡기 위해 노력했다. AI 작업에 CPU 대신 저렴한 GPU를 사용했지만, 이 접근 방식은 효율성이 떨어지는 것으로 밝혀졌다.[69]
------------------------------
2022년부터 메타는 전문 인공지능 하드웨어 및 소프트웨어 채택에서 다른 기술 기업들을 따라잡기 위해 노력했다. AI 작업에 CPU 대신 저렴한 GPU를 사용했지만, 이 접근 방식은 효율성이 떨어지는 것으로 밝혀졌다.[69]
------------------------------
